In [1]:
import yfinance as yf
import numpy as np

msft = yf.Ticker("^CMC200")

# get historical market data
hist = msft.history(period="max")[['Open']]
hist['Open'] = (np.where(hist['Open'] < 100, hist['Open'] * 10, hist['Open']))
    
    
hist.to_csv('hist.csv')

In [2]:
import requests
import pandas as pd
from datetime import datetime 

def get_fund_ROI(address, file_name):
    sent = requests.get('https://api.bloxy.info/widget/address_value_daily?address=' + address + '&currency=ETH&key=ACCunOMWYpmCp&format=table&price_currency=USD').json()
    df1 = pd.DataFrame(sent)
    df1.rename(columns={0:'Date-USD', 11: 'ROI'}, inplace=True)
    df1['ROI-USD'] = df1.ROI.cumsum()
    
    print(df1)

#     sent = requests.get('https://api.bloxy.info/widget/address_value_daily?address=' + address + '&currency=ETH&key=ACCunOMWYpmCp&format=table&price_currency=ETH').json()
#     df2 = pd.DataFrame(sent)
#     df2.rename(columns={0:'Date', 11: 'ROI'}, inplace=True)
#     df2['ROI-ETH'] = df2.ROI.cumsum()

#     df = pd.concat([df1, df2], axis=1, sort=False)[['Date', 'ROI-USD', 'ROI-ETH']]
#     df.to_csv(file_name, index=False)
    
get_fund_ROI('0xCB60D600160D005845Ec999f64266D5608fd8943', 'Fnd.csv')

       Date-USD          1          2            3            4          5  \
0    2019-10-27  29.751442  11.484415     0.000000    18.695894   0.428867   
1    2019-10-28   0.000000   0.000000    18.695894    18.966912   0.271018   
2    2019-10-29   0.000000   0.000000    18.966912    19.742468   0.775556   
3    2019-10-30  30.079268  16.427450    19.742468    33.267028  -0.127258   
4    2019-10-31   0.000000   0.000000    33.267028    33.119642  -0.147386   
..          ...        ...        ...          ...          ...        ...   
256  2020-07-09   0.000000   0.000000  2788.261628  2847.162651  58.901023   
257  2020-07-10   0.000000   0.000000  2847.162651  2796.197492 -50.965159   
258  2020-07-11   0.000000   0.000000  2796.197492  2822.776628  26.579136   
259  2020-07-12   0.000000   0.000000  2822.776628  2808.290445 -14.486183   
260  2020-07-13   0.000000   0.000000  2808.290445  2808.290445   0.000000   

               6            7            8           9         

In [40]:
sent = requests.get('https://api.bloxy.info/widget/address_value_daily?address=' + '0xCB60D600160D005845Ec999f64266D5608fd8943' + '&currency=ETH&key=ACCunOMWYpmCp&format=table&price_currency=USD').json()
df1 = pd.DataFrame(sent)
df1.rename(columns={0:'Date', 11: 'ROI'}, inplace=True)
df1['ROI-USD'] = df1.ROI.cumsum()
df1 = df1[['Date', 'ROI-USD']]
df1 = df1.set_index('Date')
df1.index = pd.to_datetime(df1.index)
print(df1)

               ROI-USD
Date                  
2019-10-27    1.441500
2019-10-28    2.891113
2019-10-29    6.980108
2019-10-30    6.724681
2019-10-31    6.281641
...                ...
2020-07-09  135.119013
2020-07-10  133.328980
2020-07-11  134.279525
2020-07-12  133.766336
2020-07-13  133.766336

[261 rows x 1 columns]


In [41]:
df = pd.concat([df1, hist], axis=1, sort=False)
df = df.interpolate(axis=0).dropna(subset=['ROI-USD'])

In [7]:
print(df)
df.to_csv('hello.csv')

               ROI-USD      Open
Date                            
2019-10-27    1.441500  174.7850
2019-10-28    2.891113  193.3600
2019-10-29    6.980108  191.3900
2019-10-30    6.724681  192.6100
2019-10-31    6.281641  187.4400
...                ...       ...
2020-07-09  135.119013  195.8200
2020-07-10  133.328980  186.0775
2020-07-11  134.279525  176.3350
2020-07-12  133.766336  166.5925
2020-07-13  133.766336  156.8500

[261 rows x 2 columns]


In [9]:
start_cmc200 = df.head(1)['Open'][0]

In [33]:
print(((df['Open'] - start_cmc200) / start_cmc200))

Date
2019-10-27    0.000000
2019-10-28    0.106273
2019-10-29    0.095002
2019-10-30    0.101982
2019-10-31    0.072403
                ...   
2020-07-09    0.120348
2020-07-10    0.064608
2020-07-11    0.008868
2020-07-12   -0.046872
2020-07-13   -0.102612
Name: Open, Length: 261, dtype: float64


In [21]:
((df['Open'] - start_cmc200)).to_csv('data.csv')

In [60]:
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()

data = cg.get_coin_market_chart_by_id('bitcoin', 'eth', 500)

dates = [datetime.fromtimestamp(data['prices'][i][0] / 1000).strftime('%Y-%m-%d') for i in range(len(data['prices']))]
price  = [data['prices'][i][1] for i in range(len(data['prices']))]

df_data = {'BTC-ETH': price, 'Date' : dates} 
df_btc = pd.DataFrame(df_data)
df_btc = df_btc.set_index('Date')
df_btc.index = pd.to_datetime(df_btc.index)

df2 = pd.concat([df, df_btc], axis=1, sort=False)
df2 = df2.dropna(subset=['ROI-USD'])

start_btc = df2.head(1)['BTC-ETH'][0]
df2['BTC-ETH'] = (((df2['BTC-ETH'] - start_btc) / start_btc)*100)

df2.drop(df2.tail(1).index,inplace=True)

df2.to_csv('btc_data', index=True)

In [61]:
print(df2)

               ROI-USD      Open    BTC-ETH
Date                                       
2019-10-27    1.441500  174.7850   0.000000
2019-10-28    2.891113  193.3600   0.428164
2019-10-29    6.980108  191.3900  -1.374407
2019-10-30    6.724681  192.6100  -4.122372
2019-10-31    6.281641  187.4400  -2.742593
...                ...       ...        ...
2020-07-08  133.006549  196.9000 -24.780848
2020-07-09  135.119013  195.8200 -25.825872
2020-07-10  133.328980  186.0775 -25.906335
2020-07-11  134.279525  176.3350 -25.214487
2020-07-12  133.766336  166.5925 -25.021140

[260 rows x 3 columns]
